In [5]:
INPUT_DIR = "../data/monin/combined_dataset"
PUSH_PATH = "ThreeBibas/recipie-mask-image"

In [7]:
!git config --global credential.helper store
!pip install -q transparent-background
from huggingface_hub import login
from secret import HF_TOKEN

login(token=HF_TOKEN, add_to_git_credential=True)

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/bulat/.cache/huggingface/token
Login successful


In [13]:
from datasets import load_dataset, Dataset
from huggingface_hub import HfApi
from transparent_background import Remover
import os
from PIL import Image

dataset = load_dataset("imagefolder", data_dir=INPUT_DIR)

mask_dir = "../data/monin/mask_dataset"
os.makedirs(mask_dir, exist_ok=True)

dataset.keys()

Resolving data files:   0%|          | 0/550 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/139 [00:00<?, ?it/s]

dict_keys(['train', 'test'])

In [ ]:
for split in dataset.keys():
    split_dir = os.path.join(mask_dir, split)
    os.makedirs(split_dir, exist_ok=True)
    
    for i, sample in enumerate(dataset[split]):
        img = sample["image"]
        
        if img.mode != 'RGB':
            img = img.convert('RGB')
        
        mask = Remover().process(img, type="map")
        mask.save(os.path.join(split_dir, f"mask_{i}.png"))

In [18]:
from tqdm import tqdm

data = []

for split in dataset.keys():
    for i, sample in tqdm(enumerate(dataset[split])):
        img = sample["image"]
        text = sample["text"]
        
        mask_path = os.path.join(mask_dir, split, f"mask_{i}.png")
        
        if os.path.exists(mask_path):
            mask = Image.open(mask_path)

            data.append({
                "text": text,
                "conditioning_image": mask,
                "image": img
            })
        else:
            print(f"Mask not found for image {i} in split {split}")

new_dataset = Dataset.from_list(data)

api = HfApi()
api.create_repo(PUSH_PATH, repo_type="dataset", private=False, exist_ok=True)
new_dataset.push_to_hub(PUSH_PATH)

549it [00:08, 63.92it/s]
138it [00:02, 63.33it/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/687 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/ThreeBibas/recipie-mask-image/commit/b5bfcfb45554d2037630c9467366a5fa295a645b', commit_message='Upload dataset', commit_description='', oid='b5bfcfb45554d2037630c9467366a5fa295a645b', pr_url=None, pr_revision=None, pr_num=None)